<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [6]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
data = pd.read_csv('reviews_vidjew_es.csv')
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [9]:
import re 

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [11]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[un, producto, perfecto, para, salir, de, casa..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, con, nintendo, switch, hay, forma, ..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, los, utilizo, para, pc, dan, nin..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[el, cable, funciona, bien, podria, ser, un, p..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [12]:
dictionary = Dictionary(data['Pre-Processed'].values)

In [13]:
dictionary.filter_extremes(no_below=5, no_above=0.5) 

In [14]:
# Corpus
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

In [16]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 7, passes=10)

In [22]:
model.print_topics(num_words=10)

[(0,
  '0.065*"de" + 0.049*"que" + 0.041*"el" + 0.035*"la" + 0.034*"en" + 0.031*"un" + 0.029*"los" + 0.028*"para" + 0.027*"con" + 0.019*"es"'),
 (1,
  '0.087*"muy" + 0.049*"la" + 0.047*"pero" + 0.043*"es" + 0.033*"se" + 0.028*"bien" + 0.023*"para" + 0.023*"bonito" + 0.021*"en" + 0.020*"los"'),
 (2,
  '0.049*"el" + 0.043*"que" + 0.038*"lo" + 0.032*"es" + 0.032*"la" + 0.025*"por" + 0.025*"un" + 0.025*"pero" + 0.025*"de" + 0.024*"se"'),
 (3,
  '0.067*"que" + 0.059*"de" + 0.043*"el" + 0.038*"lo" + 0.028*"es" + 0.026*"la" + 0.024*"se" + 0.022*"muy" + 0.019*"una" + 0.019*"un"'),
 (4,
  '0.077*"que" + 0.054*"la" + 0.049*"es" + 0.036*"de" + 0.033*"el" + 0.029*"se" + 0.028*"lo" + 0.024*"calidad" + 0.021*"ha" + 0.020*"para"'),
 (5,
  '0.051*"el" + 0.047*"de" + 0.043*"son" + 0.034*"calidad" + 0.031*"la" + 0.030*"muy" + 0.026*"pero" + 0.023*"que" + 0.021*"ha" + 0.019*"las"'),
 (6,
  '0.071*"en" + 0.069*"la" + 0.063*"el" + 0.045*"de" + 0.026*"es" + 0.023*"que" + 0.020*"muy" + 0.018*"se" + 0.016*"co

###  `[25 pts]` Punto 3: Visualización de LDA

In [18]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

In [23]:
d = dictionary.doc2bow(["calidad", "bonito", "producto"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics

[(1, 0.035757937),
 (2, 0.36250913),
 (3, 0.035846546),
 (4, 0.035825007),
 (5, 0.4582826),
 (6, 0.035878852),
 (7, 0.035899915)]

In [24]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [25]:
for t in range(0,7):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Processed'].apply(lambda doc: get_doc_top_n(doc, t))

In [26]:
for t in range(0,7):
    print(f"*********************************** TOPIC {t} ***********************************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"Comentario #{i}")
        print(articulo[:500])
        print()
    print()

*********************************** TOPIC 0 ***********************************
Comentario #0
Lo bueno Una mochila espaciosa y robusta, en el compartimento más cercano a la espalda hay espacio asignado para un joycons adicionales, dock, cargador, mando pro, y obviamente para una switch con sus joycons. En uno de los bolsillos externos tiene un espacio con cierre para poner los cartuchos sueltos. En el bolsillo medio hay espacio suficiente para un portátil de 13" y una tablet. Lo no tan bueno Grita a todas luces que dentro lleva una switch, lo bueno es que al menos el logotipo de switch q

Comentario #1
Las tapas para los joystics no están nada mal, ayudan un montón cuando el joystic original está reventado, pero si juegas al fifa con tu amigo todo motivado, son un tostón porque si haces mucha fuerza sobre el controlador para que el jugador te haga caso y puedas meterle una soba al otro, se caen. Supongo que si te gusta jugar motivado al 200% lo mejor es cambiar directamente el joystic.

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

In [ ]:
Las conclusiones que puedo sacar de los tres ultimos talleres es que el que mejores resultado fue el modelo SVM ya que no presento muchos falsos positivos,
en el modelo de codo los cluster quedaron muy dispersos no se agruparon muy bien y en este modelo de temas agrupo la mayoria de los temas en el PC1.